In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import esm
import numpy as np
import pandas as pd
import gc

In [ ]:
import pandas as pd
path_train= "/content/drive/MyDrive/ICPS/"
path_test = "/content/drive/MyDrive/ICPS/"

path_result = "/content/drive/MyDrive/ICPS/"
path_model ="/content/drive/MyDrive/ICPS/"

filetrain = "train_icp.csv"
filename_test = "test_icp.csv"
file_train = filetrain
file_test = filename_test

In [ ]:
df_train = pd.read_csv(path_train + file_train)
df_test = pd.read_csv(path_train + file_test)

In [ ]:
# Load model ESM2
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval().cuda()

In [ ]:
def extract_esm_feature(sequence):
    # Chuẩn hóa input
    data = [("protein", sequence)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        batch_tokens = batch_tokens.to("cuda")
        results = model(batch_tokens, repr_layers=[33], return_contacts=False)
        token_representations = results["representations"][33]

    feature = token_representations[0, 1:len(sequence)+1].cpu().numpy()

    if feature.shape[0] > MAX_LEN:
        feature = feature[:MAX_LEN]
    else:
        pad_len = MAX_LEN - feature.shape[0]
        feature = np.pad(feature, ((0, pad_len), (0, 0)), mode='constant')

    if feature.shape[1] > 512:
        feature = feature[:, :512]  # Cắt giảm từ 1280 xuống 512
    else:
        pad_len = 512 - feature.shape[1]
        feature = np.pad(feature, ((0, 0), (0, pad_len)), mode='constant')  # Padding nếu thiếu chiều

    # Đảm bảo chiều dài đầu vào là 1220
    if feature.shape[0] > 1220:
        feature = feature[:1220]  # Cắt nếu quá dài
    else:
        pad_len = 1220 - feature.shape[0]
        feature = np.pad(feature, ((0, pad_len), (0, 0)), mode='constant')  # Padding nếu thiếu chiều
    return feature

In [ ]:
X = []
y = []

for idx, row in df_train.iterrows():
    try:
        feat = extract_esm_feature(row['Sequence'])
        X.append(feat)
        y.append(row['Target'])

       # Thông báo sau mỗi 50 chuỗi
        if idx % 50 == 0:
            print(f"Đã xử lý {idx}/{len(df_train)} chuỗi")
    except Exception as e:
        print(f"Lỗi ở dòng {idx}: {e}")


In [ ]:
np.save("/content/drive/MyDrive/ICPS/new_train_X_esm.npy", X)
np.save("/content/drive/MyDrive/ICPS/new_train_y_esm.npy", y)


In [ ]:
X1 = []
y1 = []

for idx, row in df_test.iterrows():
    try:
        feat = extract_esm_feature(row['Sequence'])
        X1.append(feat)
        y1.append(row['Target'])

       # Thông báo sau mỗi 50 chuỗi
        if idx % 50 == 0:
            print(f"Đã xử lý {idx}/{len(df_test)} chuỗi")
    except Exception as e:
        print(f"Lỗi ở dòng {idx}: {e}")

In [ ]:
np.save("/content/drive/MyDrive/ICPS/new_test_X_esm.npy", X1)
np.save("/content/drive/MyDrive/ICPS/new_test_y_esm.npy", y1)